## 1. 환경 설정

In [1]:
# folium 설치
# !pip install folium

In [2]:
# 라이브러리 불러오기
import folium as fol
import csv
import operator as op
from openpyxl import load_workbook as lw

In [3]:
# test
fol.Map([35.143108, 129.116242])

## 2. 위치 표시하기

In [4]:
apt = fol.Map([35.143108, 129.116242], 
               zoom_start = 16)
apt_lat, apt_long = 35.143108, 129.116242
fol.Marker([35.143108, 129.116242]).add_to(apt)
apt

In [5]:
tar_lat, tar_long = 35.146117171087, 129.11531819025
marine = fol.Map([tar_lat, tar_long], zoom_start = 17)
fol.Marker([tar_lat, tar_long]).add_to(marine)
marine

### 2-1 여러 위치 표시

In [6]:
apt = fol.Map([apt_lat, apt_long], 
               zoom_start = 16)
fol.Marker([apt_lat, apt_long], popup = "Samik Apt",
          icon = fol.Icon(icon = "home")).add_to(apt)
fol.Marker([tar_lat, tar_long], tooltip = "Centre", 
           icon = fol.Icon(color = "red") ).add_to(apt)
apt

## 3. 프로젝트 표시

11장에서 사용하던 데이터에 추가로, 위치정보 데이터가 필요하다.
- 링크 : https://data.seoul.go.kr/dataList/OA-1335/S/1/datasetView.do

csv로 내려받으면 된다.

- Github : https://github.com/FATIH-BIGDATA/Gilbut_PythonProgramming

11장 코드 참고 요망

### 3-1 데이터 가져오기

In [7]:
# 인구 데이터
f = open("./data/LOCAL_PEOPLE_DONG_202206.csv", 
        encoding = 'utf8')
data = csv.reader(f)
next(data) # 컬럼명 제거
data = list(data)

# 행정동 데이터
code = lw("./data/행정동코드_매핑정보_20200325.xlsx", 
                        data_only = True) # 수식 제외 셀값만 가져오기

# 행정동코드 시트의 데이터 가져오기
code = code['행정동코드']

# 행과 열의 데이터 출력 
all_cell = []

# code 데이터를 행 단위로 가져온다.
for r in code.rows:
    r_value = [] # 빈 셀을 만든다
    
    # 행 단위로 가져온 데이터에서 셀값을 추출해 리스트에 담는다.
    for cell in r:
        r_value.append(cell.value)
    
    # 위의 과정을 반복하며 모든 데이터를 가져온다.
    all_cell.append(r_value)

# 컬럼을 제외한다.
code = all_cell[2:]

# 위경도 데이터
f2 = open("./data/서울시 행정동별 전력 사용량 2008년 위치정보 (좌표계_ WGS1984).csv", 
        encoding = 'cp949')
where = csv.reader(f2)
next(where) # 컬럼명 제거
where = list(where)

# 행정동 데이터 자료형 변환
for row in data:
    for i in range(32):
        if i == 0 :
            row[i] = str(row[i])
        elif i <= 2 :
            row[i] = int(row[i])
        else :
            row[i] = float(row[i])
            
# 코드 데이터 자료형 변환
for row in code:
    row[0], row[1] = int(row[0]), int(row[1])
    
# 위경도 데이터 자료형 변환
for row in where:
    row[2], row[-2], row[-1] = int(row[2]), float(row[-2]), float(row[-1])  

### 3-2 동별 인구수 저장 & 내림차순 정렬

In [8]:
pop = {}

for r in data:
    dong_code, p = r[2], r[3]
    
    if dong_code not in pop.keys():
        pop[dong_code] = p
    else :
        pop[dong_code] += p
        
top19 = sorted(pop.items(), key = op.itemgetter(1), reverse = True)[:19]

# 행정동 코드 저장
top_code = [c[0] for c in top19]

### 3-3 행정동명에 따라 위경도 찾기

In [9]:
top19_name = [0 for i in range(19)]
top19_lat = [0 for i in range(19)]
top19_long = [0 for i in range(19)]

for i in range(len(top19)):
    # 행정동 이름
    for r in code:
        if r[1] == top_code[i]:
            top19_name[i] = r[-1]   
    
    # 행정동 위경도
    for r in where:
        if r[3] == top19_name[i]:
            top19_lat[i], top19_long[i] = r[-1], r[-2]

print(top19_name)
print(top19_lat)
print(top19_long)

['역삼1동', '여의동', '서교동', '서초3동', '진관동', '종로1.2.3.4가동', '가산동', '신촌동', '길동', '양재1동', '역삼2동', '영등포동', '강일동', '오류2동', '한강로동', '가양1동', '대치2동', '상계6.7동', '상암동']
[37.4999211, 0, 37.5550337, 37.4856591, 37.64193, 37.5777701, 37.4756389, 37.5659638, 37.5398758, 37.4704744, 37.4984733, 37.5236448, 37.5632192, 37.4845164, 37.5299999, 37.5734646, 37.4955294, 37.6466982, 37.5727035]
[127.0367584, 0, 126.9228533, 127.0097616, 126.9387277, 126.9892737, 126.8850721, 126.9384315, 127.1473638, 127.0236975, 127.0449272, 126.9092502, 127.1725746, 126.8346877, 126.9691414, 126.8352521, 127.0662436, 127.0614074, 126.8802908]


### 3-4 7자리 행정동 코드 처리

In [10]:
for i in range(len(top19_name)):
    
    # 위경도에 0인 데이터가 있으면
    if top19_lat[i] == 0 or top19_long[i] == 0 :
        
        # 7자리 행정동 코드 저장 
        for r in code:
            if top_code[i] == r[1]:
                code7 = r[0]
                
        # 7자리 코드 위경도 구하기
        for r in where:
            if code7 == r[2]:
                top19_lat[i], top19_long[i] = r[-1], r[-2]

print(top19_lat)
print(top19_long)

[37.4999211, 37.5286662, 37.5550337, 37.4856591, 37.64193, 37.5777701, 37.4756389, 37.5659638, 37.5398758, 37.4704744, 37.4984733, 37.5236448, 37.5632192, 37.4845164, 37.5299999, 37.5734646, 37.4955294, 37.6466982, 37.5727035]
[127.0367584, 126.9290731, 126.9228533, 127.0097616, 126.9387277, 126.9892737, 126.8850721, 126.9384315, 127.1473638, 127.0236975, 127.0449272, 126.9092502, 127.1725746, 126.8346877, 126.9691414, 126.8352521, 127.0662436, 127.0614074, 126.8802908]


### 3-5 지도로 그리기

In [11]:
for i in range(len(top_code)):
    top19_name[i] = str(i + 1) + '-' + top19_name[i]
    
top19_map = fol.Map([top19_lat[0], top19_long[0]], zoom_start = 10)

for i in range(19):
    fol.Marker([top19_lat[i], top19_long[i]], tooltip = top19_name[i],
              icon = fol.Icon(color = "purple", icon_color = "yellow", icon = "star",
                             prefix = 'fa')
              ).add_to(top19_map)

top19_map

### 3-6 전체 코드 정리 (여성인구 상위 19개)

In [12]:
# 인구 데이터
f = open("./data/LOCAL_PEOPLE_DONG_202206.csv", 
        encoding = 'utf8')
data = csv.reader(f)
next(data) # 컬럼명 제거
data = list(data)

# 행정동 데이터
code = lw("./data/행정동코드_매핑정보_20200325.xlsx", 
                        data_only = True) # 수식 제외 셀값만 가져오기

# 행정동코드 시트의 데이터 가져오기
code = code['행정동코드']

# 행과 열의 데이터 출력 
all_cell = []

# code 데이터를 행 단위로 가져온다.
for r in code.rows:
    r_value = [] # 빈 셀을 만든다
    
    # 행 단위로 가져온 데이터에서 셀값을 추출해 리스트에 담는다.
    for cell in r:
        r_value.append(cell.value)
    
    # 위의 과정을 반복하며 모든 데이터를 가져온다.
    all_cell.append(r_value)

# 컬럼을 제외한다.
code = all_cell[2:]

# 위경도 데이터
f2 = open("./data/서울시 행정동별 전력 사용량 2008년 위치정보 (좌표계_ WGS1984).csv", 
        encoding = 'cp949')
where = csv.reader(f2)
next(where) # 컬럼명 제거
where = list(where)


# 행정동 데이터 자료형 변환
for row in data:
    for i in range(32):
        if i == 0 :
            row[i] = str(row[i])
        elif i <= 2 :
            row[i] = int(row[i])
        else :
            row[i] = float(row[i])
            
# 코드 데이터 자료형 변환
for row in code:
    row[0], row[1] = int(row[0]), int(row[1])
    
# 위경도 데이터 자료형 변환
for row in where:
    row[2], row[-2], row[-1] = int(row[2]), float(row[-2]), float(row[-1])  
  

# 여성인구 저장   
pop = {}

for r in data:
    dong_code, p = r[2], sum(r[18:32])
    
    if dong_code not in pop.keys():
        pop[dong_code] = p
    else :
        pop[dong_code] += p
        
top19 = sorted(pop.items(), key = op.itemgetter(1), reverse = True)[:19]

# 행정동 코드 저장
top_code = [c[0] for c in top19]


# 위경도 찾기
top19_name = [0 for i in range(19)]
top19_lat = [0 for i in range(19)]
top19_long = [0 for i in range(19)]

for i in range(len(top19)):
    # 행정동 이름
    for r in code:
        if r[1] == top_code[i]:
            top19_name[i] = r[-1]   
    
    # 행정동 위경도
    for r in where:
        if r[3] == top19_name[i]:
            top19_lat[i], top19_long[i] = r[-1], r[-2]

            
# 7자리 행정동 코드 처리
for i in range(len(top19_name)):
    
    # 위경도에 0인 데이터가 있으면
    if top19_lat[i] == 0 or top19_long[i] == 0 :
        
        # 7자리 행정동 코드 저장 
        for r in code:
            if top_code[i] == r[1]:
                code7 = r[0]
                
        # 7자리 코드 위경도 구하기
        for r in where:
            if code7 == r[2]:
                top19_lat[i], top19_long[i] = r[-1], r[-2]

                
# 지도 그리기
for i in range(len(top_code)):
    top19_name[i] = str(i + 1) + '-' + top19_name[i]
    
top19_map = fol.Map([top19_lat[0], top19_long[0]], zoom_start = 10)

# 반반 나누어 구분 표시
for i in range(19):
    if i >= 10:
        fol.Marker([top19_lat[i], top19_long[i]], tooltip = top19_name[i],
                  icon = fol.Icon(color = "purple", icon_color = "pink", icon = "female",
                                 prefix = 'fa')
                  ).add_to(top19_map)
    else:
        fol.Marker([top19_lat[i], top19_long[i]], tooltip = top19_name[i],
                  icon = fol.Icon(color = "purple", icon_color = "red", icon = "female",
                                 prefix = 'fa')
                  ).add_to(top19_map)

top19_map

## 4. Self Check

### 4-1

In [13]:
hae = fol.Map([35.15970, 129.16047], zoom_start = 11)
fol.Marker([35.15970, 129.16047], tooltip = "해운대",
          icon = fol.Icon(color = "lightblue", icon_color = "white",
                          icon = "hand-o-down",prefix = 'fa')).add_to(hae); hae

### 4-2

In [14]:
hae = fol.Map([35.15970, 129.16047], zoom_start = 11)
fol.Marker([35.15970, 129.16047], tooltip = "해운대",
          icon = fol.Icon(color = "lightblue", icon_color = "white",
                          icon = "hand-o-down",prefix = 'fa')).add_to(hae)
fol.Marker([35.23360, 129.08309], popup = "부산대",
          icon = fol.Icon(color = "darkblue", icon_color = "white",
                          icon = "university",prefix = 'fa')).add_to(hae)
fol.Marker([35.17357, 128.94632], tooltip = "김해공항",
          icon = fol.Icon(color = "green", icon_color = "white",
                          icon = "plane",prefix = 'fa')).add_to(hae); hae